# Merge NMR and MS Metabolomic Data for RCC Stage Predictions

Author: Olatomiwa Bifarin<br>
Department of Biochemistry and Molecular Biology<br>
University of Georgia<br>
Edison Lab<br>

Last edited: 10FEB2020 

 
### Notebook Outline

1. [Background](#1)
2. [Load Data](#2)
3. [Merge NMR and MS Data](#3)


In [ ]:
#import python packages and libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import style

#For Seaborn plots
import seaborn as sns; sns.set(style='white')

#To ignore warning
import warnings
warnings.filterwarnings('ignore')

# More sharp and legible graphics
%config InlineBackend.figure_format = 'retina'

# Set seaborn figure labels to 'talk', to be more visible. 
sns.set_context('talk')

## 1. Background
<a id="1"></a>

In this notebook, I merge all MS data and NMR data into one dataframe, with the appropriate RCC meta-data. 

## 2. Load Data
<a id="2"></a>

In [ ]:
dfMS = pd.read_excel('data/RCC_S280_combinedData_7097.xlsx')

In [ ]:
dfMS_trans = dfMS.T # Transpose dataframe
IDs = np.arange(1, dfMS_trans.shape[1] + 1) # Generate a range of numbers (7098) to replace nan values in the name row
IDs = pd.Series(IDs) # creating series 
dfMS_trans.iloc[4] = dfMS_trans.iloc[4].fillna(IDs) # replace nan with the IDs, and replace the name row
dfMS_trans.head()

,0,1,2,3,4,5,6,7,8,9,...,7087,7088,7089,7090,7091,7092,7093,7094,7095,7096
ID,1,2,3,4,5,6,7,8,9,10,...,7088,7089,7090,7091,7092,7093,7094,7095,7096,7097
Mode,positive,positive,positive,positive,positive,positive,positive,positive,positive,positive,...,negative,negative,negative,negative,negative,negative,negative,negative,negative,negative
Molecular Weight,186.015,113.059,113.059,234.173,290.137,314.141,125.084,143.095,198.075,171.126,...,336.051,154.048,159.976,260.137,304.096,172.047,334.083,239.079,212.054,387.18
RT [min],2.317,2.792,2.889,0.884,1.512,2.666,1.181,1.221,1.901,3.507,...,0.66,1.197,4.737,2.94,0.656,3.203,2.629,2.455,2.361,0.606
Name,1,Creatinine,Creatinine,Lidocaine,Trimethoprim,Ranitidine,"4,5-Dimethyl-2-ethyloxazole",1-Aminocyclohexanecarboxylic acid,5-Acetylamino-6-amino-3-methyluracil,Gabapentin,...,7088,7089,7090,gamma-Glutamylleucine,7092,7093,7094,7095,7096,7097


`MS-Labels` is the dataframe for just MS detected metabolites IDs and some chemical properties.

In [ ]:
MS_labels = dfMS_trans.T # Transpose data frame
MS_labels = MS_labels.filter(['ID', 'Mode', 'RT [min]', 'Name', 'Formula'], axis=1)
MS_labels.head()

,ID,Mode,RT [min],Name,Formula
0,1,positive,2.317,1,NaN
1,2,positive,2.792,Creatinine,C4 H7 N3 O
2,3,positive,2.889,Creatinine,C4 H7 N3 O
3,4,positive,0.884,Lidocaine,C14 H22 N2 O
4,5,positive,1.512,Trimethoprim,C14 H18 N4 O3


`MS` is the dataframe for the MS Peak Intensities

In [ ]:
MS = dfMS_trans.drop(['Mode', 'Molecular Weight', 'Name', 'RT [min]', 'Formula'])
MS.index.name = 'MS_ID'
MS.reset_index(inplace=True) # Converts the row names into a column.
header = MS.iloc[0] # Create a new variable called 'header' from the first row of the dataset
MS = MS[1:] # Replace the dataframe with a new one which does not contain the first row
MS.rename(columns = header, inplace=True)# Rename the dataframe's column values with the header variable
MS.rename(columns={'ID':'Sample ID'}, inplace = True) # Rename column name
# To rest the row index to start from 0
MS.index = MS.index - 1 

___Import Dataframe containing Clinical ID___

In [ ]:
path = "data/Sample_List_MS.xlsx"
fields = \
[
    'Sample ID',
    'Patient ID'
]

MSpatid = pd.read_excel(path)[fields]

`MSData` is the dataframe for the MS Peak Intensities with <mark>Patient ID</mark> and <mark>Groups</mark>

In [ ]:
MSData = pd.merge(MS, MSpatid, on='Sample ID') # Merge the two dataframe on Sample ID
MSData.head()

,Sample ID,1,2,3,4,5,6,7,8,9,...,7089,7090,7091,7092,7093,7094,7095,7096,7097,Patient ID
0,C001,7.24885e+08,1.10388e+09,5.06818e+06,4682.6,18261.9,1322.95,61663,191620,5.61943e+07,...,81645.2,501574,43882.9,235674,141900,144189,1.65926e+06,105636,76801.7,PD001
1,C002,8.39383e+08,1.87456e+09,1.87657e+09,1.69931e+08,18267.8,1674.72,227698,532662,2.03319e+08,...,242145,876199,274884,236921,167226,114217,771149,303623,37380.4,PD003
2,C003,7.65629e+08,1.67333e+09,7.47349e+06,8567.13,25730.9,3246.46,133905,495502,1.92514e+08,...,273594,946435,97021.7,839316,519959,467160,1.2216e+06,219606,6270.59,PD005
3,C004,1.25663e+08,1.1825e+09,1.17911e+09,6.72327e+06,10526.3,1076.99,129811,392377,1.16585e+08,...,308199,343794,7427.46,275350,145776,64094.9,1.7718e+06,197519,60830.9,PD012
4,C005,1.06973e+07,8.79722e+08,8.56535e+08,438385,15913.8,1008.31,48496,122696,1.10997e+07,...,39971.3,156251,718.724,199148,50598.8,107228,58713.6,71784.4,75589.5,PD014


## 3. Merge NMR and MS Data
<a id="3"></a>

In [ ]:
dfNMR = pd.read_excel('data/binned_NMRfeatures_22JAN2020.xlsx') # Import data
dfNMR.drop(['Run_ID', 'Sample_description','Yvec', 'Run_IDOrig'], axis=1, inplace=True) # drop irrelevant features
# Merge the two dataframe on Sample ID
combined = pd.merge(MSData, dfNMR, on='Patient ID')
combined.head()

,Sample ID,1,2,3,4,5,6,7,8,9,...,hypoxanthine_1,hypoxanthine_2,formate,unk10,Trigonelline_1,Trigonellinamide_1,Trigonellinamide_2,Trigonelline_2,Trigonellinamide_3,unk11
0,C001,7.24885e+08,1.10388e+09,5.06818e+06,4682.6,18261.9,1322.95,61663,191620,5.61943e+07,...,0.019153,0.014217,0.035454,0.001337,0.017210,0.006084,0.005395,0.008883,0.003132,0.001725
1,C002,8.39383e+08,1.87456e+09,1.87657e+09,1.69931e+08,18267.8,1674.72,227698,532662,2.03319e+08,...,0.013937,0.023921,0.022766,0.002879,0.034096,0.011080,0.013063,0.022937,0.010969,0.000611
2,C003,7.65629e+08,1.67333e+09,7.47349e+06,8567.13,25730.9,3246.46,133905,495502,1.92514e+08,...,0.008652,0.007889,0.021506,0.018444,0.193980,0.007303,0.007281,0.101210,0.007860,0.001027
3,C004,1.25663e+08,1.1825e+09,1.17911e+09,6.72327e+06,10526.3,1076.99,129811,392377,1.16585e+08,...,0.011827,0.019886,0.031489,0.008745,0.087607,0.006037,0.006090,0.049477,0.005081,0.000741
4,C005,1.06973e+07,8.79722e+08,8.56535e+08,438385,15913.8,1008.31,48496,122696,1.10997e+07,...,0.018709,0.021821,0.035003,0.001863,0.006908,0.010074,0.009497,0.007540,0.007582,0.000976


Non-metabolic features in the `combined` dataframe include the following: 
-  Sample ID
-  Patient ID
-  Collection
-  Gender
-  Race
-  BMI
-  Smoker
-  Age
-  Groups

The shape of the MS, NMR, and the combined dataframe

In [ ]:
print(dfNMR.shape)
print (MSData.shape)
print (combined.shape)

(284, 56)
(280, 7099)
(256, 7154)


In [ ]:
dfRCC = pd.read_excel('data/RCC_MetaData_01JUL2019.xlsx')
dfRCC = dfRCC.filter(['Patient ID', 'Metastatic', 'SubTypes', 
                      'Nuclear Grade', 'Grade', 'Tstage', 
                      'Tstage Group', 'Nstage', 'Mstage', 
                      'Stage', 'Grouped Stage','Tumor Width'], axis=1)

In [ ]:
RCCdataframe = pd.merge(combined, dfRCC, on='Patient ID') # Merge the two dataframe on Sample ID
print (RCCdataframe.shape)
RCCdataframe.head()

(82, 7165)


,Sample ID,1,2,3,4,5,6,7,8,9,...,SubTypes,Nuclear Grade,Grade,Tstage,Tstage Group,Nstage,Mstage,Stage,Grouped Stage,Tumor Width
0,C001,7.24885e+08,1.10388e+09,5.06818e+06,4682.6,18261.9,1322.95,61663,191620,5.61943e+07,...,Pure Clear Cell,2.0,Low,T1a,T1,N0,M0,1.0,Early,2.9
1,C002,8.39383e+08,1.87456e+09,1.87657e+09,1.69931e+08,18267.8,1674.72,227698,532662,2.03319e+08,...,Pure Clear Cell,2.0,Low,T1a,T1,NX,NaN,NaN,NaN,3.0
2,C003,7.65629e+08,1.67333e+09,7.47349e+06,8567.13,25730.9,3246.46,133905,495502,1.92514e+08,...,Chromophobe,NaN,NaN,T1b,T1,NX,M0,NaN,NaN,5.5
3,C004,1.25663e+08,1.1825e+09,1.17911e+09,6.72327e+06,10526.3,1076.99,129811,392377,1.16585e+08,...,Pure Clear Cell,3.0,High,T1b,T1,NX,NaN,NaN,NaN,4.7
4,C005,1.06973e+07,8.79722e+08,8.56535e+08,438385,15913.8,1008.31,48496,122696,1.10997e+07,...,Pure Clear Cell,2.0,Low,T1a,T1,NX,NaN,NaN,NaN,2.4


save RCCdataframe. 

In [ ]:
# RCCdataframe.to_excel('RCCdataframe.xlsx')